In [30]:
import pandas as pd
idx = pd.IndexSlice

* Video (at least the 20s test) is height 540, width 960...

In [2]:
raw_tracks = pd.read_csv('../data/converted_3.mp4_classes.csv').set_index('frame_index')

In [79]:
# calculate centroids from bounding box
raw_tracks['centroid_x'] = (raw_tracks['bbox0'] + raw_tracks['bbox2']) / 2
raw_tracks['centroid_y'] = (raw_tracks['bbox1'] + raw_tracks['bbox3']) / 2

In [9]:
##TODO dict of appropriate values for each input file...
##filename: (x_ped_threshold, y_car_threshold)
x_y_cuts = {'converted_3.mp4':(510, 400)}

##TODO will eventually need dict of fps if they differ for time calculations...
video_fps = 4

In [10]:
## use x=510 centroid for peds
## use y=400 for cars passing

In [44]:
crossing_tracks = pd.DataFrame()

x_thr = x_y_cuts['converted_3.mp4'][0]
y_thr = x_y_cuts['converted_3.mp4'][1]

for track_id in raw_tracks['track_id'].unique():
    this_track = raw_tracks[raw_tracks['track_id'] == track_id]
    if (this_track['class_name'] == 'person').all():
        if (this_track['centroid_x'] < x_thr).any() and (this_track['centroid_x'] > x_thr).any():
            crossing_tracks = crossing_tracks.append(this_track)
    elif (this_track['class_name'] == 'car').all():
        if (this_track['centroid_y'] < y_thr).any() and (this_track['centroid_y'] > y_thr).any():
            crossing_tracks = crossing_tracks.append(this_track)
crossing_tracks = crossing_tracks.set_index('track_id', append=True)

In [47]:
crossing_tracks

,,class_name,bbox0,bbox1,bbox2,bbox3,centroid_x,centroid_y
frame_index,track_id,,,,,,,
8,16,car,1162,376,1230,415,1196.0,395.5
9,16,car,1165,377,1232,415,1198.5,396.0
10,16,car,1164,377,1233,416,1198.5,396.5
11,16,car,1164,378,1231,417,1197.5,397.5
12,16,car,1164,379,1230,417,1197.0,398.0
...,...,...,...,...,...,...,...,...
3312,1878,car,790,303,1196,506,993.0,404.5
3313,1878,car,839,304,1240,499,1039.5,401.5
3314,1878,car,858,297,1281,506,1069.5,401.5


In [53]:
# add column denoting crossing frames

with_crossing = pd.DataFrame()

for frame, track_df in crossing_tracks.groupby(level=1):
    if (track_df['class_name'] == 'person').all():
        track_df['x_boundary_dist'] = abs(track_df['centroid_x'] - x_thr)
        track_df['crossing_frame'] = track_df['x_boundary_dist'] == track_df['x_boundary_dist'].min()
    elif (track_df['class_name'] == 'car').all():
        track_df['y_boundary_dist'] = abs(track_df['centroid_y'] - y_thr)
        track_df['crossing_frame'] = track_df['y_boundary_dist'] == track_df['y_boundary_dist'].min()

    with_crossing = with_crossing.append(track_df)
with_crossing = with_crossing.drop(columns=['bbox0', 'bbox1', 'bbox2', 'bbox3'])

In [54]:
with_crossing

,,class_name,centroid_x,centroid_y,y_boundary_dist,crossing_frame,x_boundary_dist
frame_index,track_id,,,,,,
8,16,car,1196.0,395.5,4.5,False,NaN
9,16,car,1198.5,396.0,4.0,False,NaN
10,16,car,1198.5,396.5,3.5,False,NaN
11,16,car,1197.5,397.5,2.5,False,NaN
12,16,car,1197.0,398.0,2.0,False,NaN
...,...,...,...,...,...,...,...
3312,1878,car,993.0,404.5,4.5,False,NaN
3313,1878,car,1039.5,401.5,1.5,False,NaN
3314,1878,car,1069.5,401.5,1.5,False,NaN


In [61]:
# reformat df to 1 row per crossing, with relevant info

crossing_info = pd.DataFrame()

for frame, track_df in with_crossing.groupby(level=1):
    crossing = {}
    crossing['first_frame'] = track_df.index.get_level_values('frame_index').min()
    crossing['last_frame'] = track_df.index.get_level_values('frame_index').max()
    crossing['crossed_frame'] = track_df[track_df['crossing_frame'] == True].index.get_level_values('frame_index')
    crossing['frames_before_cross'] = crossing['crossed_frame'] - crossing['first_frame']
    crossing['class_name'] = track_df['class_name'].iloc[0]
    crossing['track_id'] = track_df.index.get_level_values('track_id')[0]
    
    crossing_info = crossing_info.append(pd.DataFrame(crossing).set_index('track_id'))

In [62]:
crossing_info

,first_frame,last_frame,crossed_frame,frames_before_cross,class_name
track_id,,,,,
16,8,169,16,8,car
16,8,169,25,17,car
19,10,184,19,9,car
28,13,204,195,182,car
28,13,204,196,183,car
...,...,...,...,...,...
1802,3112,3116,3113,1,car
1806,3118,3144,3119,1,car
1832,3177,3214,3209,32,car


In [76]:
crossing_info[crossing_info['class_name'] == 'person']

,first_frame,last_frame,crossed_frame,frames_before_cross,class_name
track_id,,,,,
854,1492,1517,1499,7,person
1080,1839,1935,1892,53,person


In [78]:
## in progress, count cars crossing before each ped does

crossing_info[(crossing_info['class_name'] == 'car') & (
            crossing_info['crossed_frame'].isin(range(1839, 1892)))] ## range should be (first_frame, crossed_frame) of relevant ped

,first_frame,last_frame,crossed_frame,frames_before_cross,class_name
track_id,,,,,
